In [1]:
from nowcasting_dataset.data_sources import NWPDataSource
from pathlib import Path
import pandas as pd

In [2]:
BUCKET = Path('solar-pv-nowcasting-data')
NWP_BASE_PATH = BUCKET / 'NWP/UK_Met_Office/UKV_zarr'

In [3]:
nwp_ds = NWPDataSource(
    filename=f'gs://{NWP_BASE_PATH}',
    history_len=0,
    forecast_len=1,
    image_size_pixels=2,
    meters_per_pixel=2_000)

In [4]:
%%time
nwp_ds.open()

CPU times: user 6.58 s, sys: 275 ms, total: 6.86 s
Wall time: 10 s


In [5]:
%%time
datetimes = nwp_ds.datetime_index()

CPU times: user 13.3 ms, sys: 3.7 ms, total: 17 ms
Wall time: 15.1 ms


In [6]:
nwp_ds.data

,Array,Chunk
Bytes,5.65 TiB,117.73 MiB
Shape,"(10, 5442, 37, 704, 548)","(1, 2, 20, 704, 548)"
Count,300922 Tasks,54460 Chunks
Type,float64,numpy.ndarray


In [7]:
%%time
example = nwp_ds.get_example(
    x_meters_center=100000,
    y_meters_center=100000,
    t0_dt=pd.Timestamp('2019-11-16 10:05:00')
)


CPU times: user 3.08 s, sys: 1.06 s, total: 4.14 s
Wall time: 1.52 s


In [8]:
%%timeit
example = nwp_ds.get_example(
    x_meters_center=552333.2498806263,
    y_meters_center=120659.03637486309,
    t0_dt=pd.Timestamp('2019-01-16 14:15:00')
)

8.67 ms ± 391 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
datetimes[70:]

DatetimeIndex(['2018-01-01 05:50:00', '2018-01-01 05:55:00',
               '2018-01-01 06:00:00', '2018-01-01 06:05:00',
               '2018-01-01 06:10:00', '2018-01-01 06:15:00',
               '2018-01-01 06:20:00', '2018-01-01 06:25:00',
               '2018-01-01 06:30:00', '2018-01-01 06:35:00',
               ...
               '2019-12-31 22:15:00', '2019-12-31 22:20:00',
               '2019-12-31 22:25:00', '2019-12-31 22:30:00',
               '2019-12-31 22:35:00', '2019-12-31 22:40:00',
               '2019-12-31 22:45:00', '2019-12-31 22:50:00',
               '2019-12-31 22:55:00', '2019-12-31 23:00:00'],
              dtype='datetime64[ns]', length=195831, freq=None)

In [16]:
target_times_hourly = pd.date_range('2018-01-01 06:00:00', '2018-01-01 07:00:00', freq='H')

In [17]:
target_times_hourly

DatetimeIndex(['2018-01-01 06:00:00', '2018-01-01 07:00:00'], dtype='datetime64[ns]', freq='H')

In [18]:
init_times = nwp_ds.data.sel(init_time=target_times_hourly, method='ffill').init_time.values

In [19]:
init_times

array(['2018-01-01T06:00:00.000000000', '2018-01-01T06:00:00.000000000'],
      dtype='datetime64[ns]')

In [20]:
import numpy as np

In [36]:
indexes = np.searchsorted(nwp_ds.data.init_time, target_times_hourly, side='right')
indexes -= 1

In [37]:
nwp_ds.data.init_time.values[indexes]

array(['2018-01-01T06:00:00.000000000', '2018-01-01T06:00:00.000000000'],
      dtype='datetime64[ns]')

In [26]:
ts.to_numpy()

numpy.datetime64('2018-01-01T06:00:00.000000000')